In [1]:
import cooler
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Input data to run this notebook

In [ ]:
# Merged Hi-C data
# Run 1_merge_Hi-C.sh script to obtain these maps
clr_neur_path = '../data/NeuNplus.sampled.1000.mcool::resolutions/250000'

neuronal_dot_cent_path = ''

h3k9me3_astro_path = ''
h3k9me3_fetal_path = ''

# Average signal curve calculation and plotting (Figure 4G)

In [2]:
def pylap_new(path_to_points,path_to_data,nbins,chrs):
    Q=0
    points=pd.read_csv(path_to_points,sep='\t',header=None)
    chr_length={'chr1':248956422,'chr2':242193529,'chr3':198295559,'chr4':190214555,'chr5':181538259,'chr6':170805979,'chr7':159345973,'chr8':145138636,'chr9':138394717,
    'chr10':133797422,'chr11':135086622,'chr12':133275309,'chr13':114364328,'chr14':107043718,'chr15':101991189,'chr16':90338345,
    'chr17':83257441,'chr18':80373285,'chr19':58617616,'chr20':64444167,'chr21':46709983,'chr22':50818468,'chrX':156040895,'chrY':57227415,'chrM':16569}
    
    points.columns=['chrom','start','end']

    for i in list(chr_length.keys()):

        points=points.drop(points[(points['chrom']==i) & (points['start']>chr_length[i]-2500000)].index)

    h=points[points['start']>=3000000]

    h.sort_values(['chrom','start'],inplace=True)

    points=pybedtools.BedTool.from_dataframe(h)
    path=pd.read_csv(path_to_data,sep='\t',header=None)
    data=pybedtools.BedTool.from_dataframe(path)
    t=data.intersect(points,wao=True).to_dataframe()

    t= t.loc[t['chrom'].isin(chrs)]
    t.loc[t.name=='.','name']=np.nan
    q=sum(t['strand']!=-1)

    n=np.zeros((q,2*nbins+1))
    k=0
    for i in chrs:
       
        tmp=t[t['chrom']==i]
       
        tmp=tmp.reset_index(drop=True)
        for j in range(len(tmp)):

            if tmp.loc[j]['strand']!=-1:
                
                n[k]=tmp.loc[j-nbins:j+nbins]['name']

                k+=1
            
    A=pd.DataFrame(n)

    return(A)

In [ ]:
def plott(xx,lab,m):
    l=[]
    u=[]
    for i in xx.columns:
        a=xx[i].dropna()
        x,y=scipy.stats.t.interval(0.95, len(a)-1, loc=np.nanmean(a,axis=0)-xx.mean(axis=0)[m], scale=scipy.stats.sem(a))
        l.append(x)
        u.append(y)
    xfit = np.linspace(0,2*m, 100) 
    
    spl = scipy.interpolate.make_interp_spline(np.arange(2*m+1),np.nanmean(xx,axis=0)-xx.mean(axis=0)[m], k=3)  
    yfit = spl(xfit)
    spl = scipy.interpolate.make_interp_spline(np.arange(2*m+1),l, k=3) 
    fit_l = spl(xfit)

    spl = scipy.interpolate.make_interp_spline(np.arange(2*m+1),u, k=3) 
    fit_u = spl(xfit)

    plt.plot(xfit, yfit, '-',label=lab)

    plt.fill_between(xfit, fit_l, fit_u,
                 color='gray', alpha=0.2)

In [ ]:
c=cooler.Cooler(clr_neur_path)
po1=pylap_new('.../polycomb_dot_anchors.hand_made_centers.bed',
             '.../astro_2replicates.bedgraph',20,c.chromnames[0:-2])
po2=pylap_new('.../polycomb_dot_anchors.hand_made_centers.bed',
             '.../fetal_2replicates.bedgraph',20,c.chromnames[0:-2])
 

plott(po1,'astrocytes',20,sns.color_palette()[1])
plott(po2,'fetal brain',20,sns.color_palette()[4])

plt.ylabel('H3K9me3 FC signal to control')
plt.legend()
plt.xticks(np.arange(0,41,10),np.arange(-200,201,100))
plt.xlabel('distance from center of contact anchor, kb')
# plt.savefig('.../H3K9me3_in_astrocytes_and_fetal_brain_loops.pdf')